# Bayesian Network Probability Density Map Generation

Generate the PDM for a SAR scenario using the technique outlined by Lin and Goodrich[1] using Bayesian Networks

[1]: L. Lin and M. A. Goodrich, ‘A Bayesian approach to modeling lost person behaviors based on terrain features in Wilderness Search and Rescue’, Comput Math Organ Theory, vol. 16, no. 3, pp. 300–323, Jul. 2010, doi: [10.1007/s10588-010-9066-2](https://doi.org/10.1007/s10588-010-9066-2).

In [ ]:
import sys, os

sys.path.insert(0, os.path.join(os.getcwd(), "../src")) # run f

In [ ]:
!pip install pomegranate

In [ ]:
from pomegranate import DiscreteDistribution, Node, BayesianNetwork, ConditionalProbabilityTable

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Install from package at https://github.com/iwishiwasaneagle/jsim/releases/latest or add use it from within the project
from jsim.Environment import Environment, HexEnvironment
from jsim.Environment.HexEnvironment.HexCoords import OffsetCoord
from jsim.Simulation import Simulation
from jsim.Meta import Action, State

In [ ]:
class BNHexEnvironment(HexEnvironment):
    def reset(self) -> State:
        pass

    def step(self, pa: Action) -> tuple[State, float]:
        pass

In [ ]:
pdm = BNHexEnvironment(m=9)

polygons = pdm.as_mpl_polygons()

fig, ax = plt.subplots(1)
ax.set_aspect('equal')

for p in polygons:
    ax.add_patch(p)

pos = OffsetCoord(col=2,row=3)
for p in HexEnvironment.offsets_to_mpl_polygons(HexEnvironment.neighbors_coord(pos)):
    ax.add_patch(p)

ax.add_patch(HexEnvironment.offset_to_mpl_polygon(pos,(1.,1.,1.,1.)))
plt.autoscale(enable=True)
plt.show()

In [ ]:
vegetation = DiscreteDistribution({'00': 0.6, '01': 0.25, '02': 0.15, '10': 0.5, '11': 0.3, '12': 0.2, '20': 0.4, '21': 0.4, '22': 0.2})
topography = DiscreteDistribution({'00': 0.6, '01': 0.25, '02': 0.15, '10': 0.5, '11': 0.3, '12': 0.2, '20': 0.4, '21': 0.4, '22': 0.2})
slope = DiscreteDistribution({'0': 0.5, '1': 0.1, '2': 0.5})

s1 = Node(vegetation, name="vegetation")
s2 = Node(topography, name="topography")
s3 = Node(slope, name="slope")

network = BayesianNetwork("PDM Agent")
network.add_states(s1, s2, s3)
network.add_edge(s1, s2)
network.add_edge(s2, s3)
network.bake()